# Modelling
<u>Tests using the following models :</u>
* Linear regression
* Random forest regressor
* Ridge and Lasso Regularization (add on to linear modelling?)

<u> Tests using the following variables:</u>
* Weather variables (rain, temperature, windspeed)
* Time variables (Day of week, month, year, time of day, public holiday)
* Sensor environment variables:
    * Sensor_id
    * Betweenness of the street 
    * Buildings in proximity to the sensor
    * Landmarks in proximity to the sensor  
    * Furniture in proximity to the sensor    
    * Lights in proximity to the sensor   


Normalise variables: should this be with MinMax or StandardScaler??

In [3]:
import copy
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, mean_squared_error,r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import time as thetime
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier, XGBRegressor
from time import time

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

import multiprocessing

# To display tables in HTML output
from IPython.display import HTML, display

from Functions import *

In [59]:
def run_model_with_cv(model, metrics, cv):
  
    scores = cross_validate(model, Xfull, Yfull, cv=cv, scoring=metrics ,return_estimator=True)

    mae_scores = scores['test_neg_mean_absolute_error']
    r2_scores = scores['test_r2']
    rmse_scores = scores['test_neg_root_mean_squared_error']

    # Report the mean scores
    print("Mean mae of %0.2f with a standard deviation of %0.2f" % (mae_scores.mean(), mae_scores.std()))
    print("Mean r2 of %0.2f with a standard deviation of %0.2f" % (r2_scores.mean(), r2_scores.std()))
    print("Mean rmse of %0.2f with a standard deviation of %0.2f" % (rmse_scores.mean(), rmse_scores.std())) 

metrics = ['neg_mean_absolute_error', 'r2', 'neg_root_mean_squared_error']
cv = KFold(n_splits=10, random_state=1, shuffle=True)
run_model_with_cv(rf_model, metrics, cv)   

Mean mae of -90.57 with a standard deviation of 0.82
Mean r2 of 0.88 with a standard deviation of 0.00
Mean rmse of -173.42 with a standard deviation of 1.97


In [57]:
def run_model_with_cv(model,model_name, metrics, cv, Xfull, Yfull, regex_name, regex_pattern):
    print("Running {} model, variables include {}".format(model_name,  regex_name))

    # Filter columns using the regex pattern in function input
    Xfull = Xfull[Xfull.columns.drop(list(Xfull.filter(regex=regex_pattern)))].copy()
    
    # Perform cross validation, time how long it takes
    start = time()
    scores = cross_validate(model, Xfull, Yfull, cv=cv, scoring=metrics ,return_estimator=False, error_score="raise")
    end = time()
    
    #  Create a dataframe containng scores for each performance metric
    df =pd.DataFrame({'mae': round(abs(scores['test_neg_mean_absolute_error'].mean()),2), 
         'r2': round(abs(scores['test_r2'].mean()),2), 'rmse': round(abs(scores['test_neg_root_mean_squared_error'].mean()),2)},
                     index =["{}_{}".format(model_name, regex_name)])
    
    print('Ran in {} minutes'.format(round((end - start)/60),2))
    return [scores, df]

### Read in formatted data

In [15]:
data = pd.read_csv("formatted_data_for_modelling.csv", index_col = False)

### Keep only sensors with relatively complete data

In [16]:
### Filter to include just sensors which we know have quite complete data 
data = data[data['sensor_id'].isin([2,6,9,10,14,18])]
data.reset_index(inplace=True, drop = True)

In [17]:
# data = data.drop(['Pressure', 'Humidity'],axis=1) # seem obviously irrelevant
data = data.drop(['sensor_id'],axis=1) # don't want this included
# Get rid of columns in which none of the sensors have a value
for column in data.columns:
    if np.nanmax(data[column]) ==0:
        del data[column]

## Prepare data for modelling - split into predictor/predictand variables

In [50]:
# The predictor variables
Xfull = data.drop(['hourly_counts'], axis =1)
# Xfull['random'] = np.random.random(size=len(Xfull))

# The variable to be predicted
Yfull = data['hourly_counts'].values

# Split data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(Xfull, Yfull, test_size=0.6666, random_state=123)

#### Standardize both training and testing data
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# Xfull = pd.DataFrame(scaler.fit_transform(Xfull), columns=Xfull.columns)

# Get list of all features
feature_list = list(Xfull.columns)

## Define models (linear regression, random forest and XGBoost)

In [51]:
lr_model = LinearRegression()
rf_model = RandomForestRegressor(n_estimators = 100, random_state = 1, n_jobs = 200)
xgb_model = XGBRegressor(random_state=1, n_jobs = 200)

## Run models with cross-validation

#### Define the error metrics for the cross-validation to return, and the parameters of the cross validatio

In [52]:
error_metrics = ['neg_mean_absolute_error', 'r2', 'neg_root_mean_squared_error']
cv_parameters = KFold(n_splits=10, random_state=1, shuffle=True)

#### Define regex's to remove columns not needed in various splits of removing column

In [53]:
column_regex_dict = {'nosubtyes':'buildings_|furniture_|landmarks_|sensor_id',
                     'withsubtypes':'buildings$|furniture$|landmarks$',
                     'time_and_weather':'buildings|furniture|landmarks|h_|lights|avg_n_floors|betweenness',
                      'just_location_features':'buildings$|furniture$|landmarks$|school_holiday|public_holiday|Temp|Humidity|Pressure|Rain|WindSpeed|Sin|Cos'}

#### Loop through each combination of the models, and the variables to include in the modelling

In [54]:
totals_df = pd.DataFrame()
models_dict = {"linear_regression": lr_model}#, "rf_regression": rf_model, "xgboost":xgb_model}
for model_name,model in models_dict.items():
    for regex_name, regex in column_regex_dict.items():
        scores, df = run_model_with_cv(model, model_name, error_metrics, cv_parameters, Xfull, Yfull, regex_name, regex)   
        totals_df = totals_df.append(df)

Running linear_regression model, variables include nosubtyes
Ran in 0 minutes
Running linear_regression model, variables include withsubtypes
Ran in 0 minutes
Running linear_regression model, variables include time_and_weather
Ran in 0 minutes
Running linear_regression model, variables include just_location_features
Ran in 0 minutes


In [60]:
totals_df

,mae,r2,rmse
linear_regression_nosubtyes,264.22,0.45,363.75
linear_regression_withsubtypes,260.30,0.48,354.47
linear_regression_time_and_weather,305.05,0.25,425.36
linear_regression_just_location_features,332.51,0.18,443.75


#### Inspect table of results for each model

In [55]:
rf_model = RandomForestRegressor(n_estimators = 100, random_state = 1, n_jobs = 32)
test_Xfull = Xfull[Xfull.columns.drop(list(Xfull.filter(regex=regex)))].copy()
output = cross_validate(rf_model, test_Xfull, Yfull, cv=6, scoring = error_metrics, return_estimator =True)

In [46]:
feature_importances = pd.DataFrame(index =[test_Xfull.columns])
feature_importances
for idx,estimator in enumerate(output['estimator']):
        print(idx, estimator)
        print("Features sorted by their score for estimator {}:".format(idx))
        feature_importances['Estimator{}'.format(idx)] = output['estimator'][idx].feature_importances_

0 RandomForestRegressor(n_jobs=32, random_state=1)
Features sorted by their score for estimator 0:
1 RandomForestRegressor(n_jobs=32, random_state=1)
Features sorted by their score for estimator 1:
2 RandomForestRegressor(n_jobs=32, random_state=1)
Features sorted by their score for estimator 2:
3 RandomForestRegressor(n_jobs=32, random_state=1)
Features sorted by their score for estimator 3:
4 RandomForestRegressor(n_jobs=32, random_state=1)
Features sorted by their score for estimator 4:
5 RandomForestRegressor(n_jobs=32, random_state=1)
Features sorted by their score for estimator 5:


In [56]:
output

{'fit_time': array([2.4244895 , 2.3930738 , 2.46689463, 2.54966235, 2.49546957,
        2.64225554]),
 'score_time': array([0.12074709, 0.12436295, 0.11818433, 0.11742091, 0.11686015,
        0.11740708]),
 'estimator': [RandomForestRegressor(n_jobs=32, random_state=1),
  RandomForestRegressor(n_jobs=32, random_state=1),
  RandomForestRegressor(n_jobs=32, random_state=1),
  RandomForestRegressor(n_jobs=32, random_state=1),
  RandomForestRegressor(n_jobs=32, random_state=1),
  RandomForestRegressor(n_jobs=32, random_state=1)],
 'test_neg_mean_absolute_error': array([-343.45321638, -342.74746853, -361.57984626, -370.50209488,
        -320.52340468, -235.88417276]),
 'test_r2': array([0.14721438, 0.16981377, 0.17785779, 0.16294626, 0.14791257,
        0.21397742]),
 'test_neg_root_mean_squared_error': array([-456.59444171, -448.86602164, -472.54867005, -486.23835702,
        -442.49006172, -348.44283189])}

In [48]:
feature_importances.sort_values('Estimator0', ascending = False)

,Estimator0,Estimator1,Estimator2,Estimator3,Estimator4,Estimator5
furniture_Bollard,0.147015,1.204725e-01,1.087627e-01,0.111370,1.498835e-01,1.582828e-01
lights,0.142412,1.337089e-01,1.222299e-01,0.124462,1.650250e-01,1.814992e-01
avg_n_floors,0.134686,3.102368e-01,3.606356e-01,0.268764,1.642069e-01,1.572662e-01
buildings_Public Display Area,0.125821,1.096132e-01,1.087207e-01,0.110957,1.306394e-01,1.216503e-01
furniture_Litter Bin,0.103737,9.722392e-02,9.640284e-02,0.098315,1.064419e-01,1.089094e-01
furniture_Bicycle Rails,0.088222,7.858071e-02,7.708186e-02,0.079483,8.300446e-02,8.685467e-02
betweenness,0.039230,9.747178e-03,1.031874e-03,0.000260,3.891647e-02,4.089956e-02
buildings_Residential,0.038837,1.156391e-02,3.549753e-04,0.075592,3.916777e-02,3.865983e-02
buildings_Storage,0.038744,2.163528e-02,2.692535e-02,0.037415,2.762493e-02,5.868574e-02
buildings_Retail,0.036517,1.861030e-02,9.239207e-04,0.000739,1.165778e-02,8.099739e-03


### Find the best model  
Use k-fold cross validation to evaluate a range of regression algorithms on the training data. Use a pipeline for evaluation which first scales the (weather) data. Print the results and assess which models perform best.

The following models were trialled:

* Decision Tree
* Random Forest
* Extra Trees
* Dummy Regressor
* Elastic Net CV
* Passive Aggressive
* RANSAC
* SGD
* TheilSen (dropped in code below because it takes too long)
* K Neighbours
* LinearRegression
* XGBoost

In [ ]:
# # Define a list of all the models to use
# Models = {'LinearRegression': LinearRegression,'DecisionTree' : DecisionTreeRegressor,
#           'RandomForest': RandomForestRegressor, 'ExtraTrees' : ExtraTreesRegressor,
#           'DummyRegressor' :DummyRegressor, 'ElasticNetCV' : ElasticNetCV, 
#           'PassiveAggressive' : PassiveAggressiveRegressor, #RANSAC': RANSACRegressor, # This one is terrible too
#           'SGD': SGDRegressor, #'TheilSen': TheilSenRegressor, # Drop this - it isn't great and takes too long
#           'KN': KNeighborsRegressor}#, 'XGBoost': xgb.XGBRegressor}
 
# # Now just run each model, but do this in multiple processes simultaneously to save time    
# # Now call that function simultaneously for each model
# p = Pool(processes=None) # A pool of processes (one for each core)
# results = p.map(run_model, [(name, model_type) for name, model_type in Models.items()])

# # Sort the results by median mse (that's item 5 in the tuple)
# results.sort(key=lambda x: x[5], reverse=True)

# # Put the results in a nice dictionary and print them
# results_dict = {}
# txt = "<table><thead><td>Name</td><td>Median R2</td><td>Median MSE</td><td>runtime (sec)</td></thead>"
# for name, model, all_r2, r2, all_mse, mse, runtime in results:
#     txt += "<tr><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>".format(name, r2, mse, runtime)
#     results_dict[name] = (model, all_r2, r2, all_mse, mse, runtime)
# txt += "</table>"
# display(HTML(txt)) # print as html

# min_mse = min([mse for (name, model, all_r2, r2, all_mse, mse, runtime) in results])
               
# x =  [ name for (name, model, all_r2, r2, all_mse, mse, runtime) in results]
# y1 = [ mse-min_mse   for (name, model, all_r2, r2, all_mse, mse, runtime) in results]
# y2 = [ r2 if r2 > 0 else 0 for (name, model, all_r2, r2, all_mse, mse, runtime) in results]

# fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(15, 7))

# ax1.set_title("MSE")
# #ax1.invert_yaxis()
# ax1.bar(range(len(x)), y1)
# ax1.set_xticks(range(len(x)))
# ax1.set_xticklabels(x, rotation=90)
# ax1.set_ylim([27000000000, 29000000000])

# ax2.set_title("R^2")
# ax2.bar(range(len(x)), y2)
# ax2.set_xticks(range(len(x)))
# ax2.set_xticklabels(x, rotation=90)

# plt.show()

# #del x,y1, y2

# ## Set up a dictionary containing the hyperparameters we want to tune
# hyperparameters_rf = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
#                   'randomforestregressor__max_depth': [None, 5, 3, 1]}
# # hyperparameters_xgb = {'xgbregressor__max_depth': range(1, 11, 2),
# #                    'xgbregressor__n_estimators' : range(50, 400, 50),
# #                    'xgbregressor__learning_rate' : [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]}
# hyperparameters_lr = {}

# # Set up the pipeline containing the scalers
# pipeline_rf = make_pipeline(MinMaxScaler(feature_range = (0,1)), 
#                          RandomForestRegressor(n_estimators=100))
# # pipeline_xgb = make_pipeline(MinMaxScaler(feature_range = (0,1)),
# #                          xgb.XGBRegressor(n_estimators=100))
# pipeline_lr = make_pipeline(MinMaxScaler(feature_range = (0,1)),
#                          LinearRegression())

# # Store the scores in a results dictionary (and print them)
# final_results = {}
# for model_values in [(pipeline_rf,  hyperparameters_rf,  'RandomForest'),
# #                      (pipeline_xgb, hyperparameters_xgb, 'XGBoost'),
#                      (pipeline_lr,  hyperparameters_lr,  'LinearRegression')]:
    
#     clf = GridSearchCV(model_values[0], model_values[1], 
#                        #cv = None, # Cross-validation method. None means default (3-fold)
#                        cv = 10, # positive intiger means k-fold (e.g. 10-fold)
#                        #scoring  = 'neg_mean_squared_error', # MSE to calculate score
#                        scoring  = 'r2', # MSE to calculate score
#                        n_jobs=multiprocessing.cpu_count()) # Run on multiple cores
    
#     #clf = GridSearchCV(model_values[0], model_values[1], cv = 10, scoring  = 'r2')
#     clf.fit(X_validate, Y_validate)
#     name = model_values[2]
#     final_results[name] = clf
#     print ("Hyperparameter results for {}".format(name))
#     print ("\tBest Score: {}".format(clf.best_score_))
#     print ("\tBest params: {}".format(clf.best_params_))